In [1]:
# Note using suppress to reduce constant red warning boxes
suppressPackageStartupMessages(
    {suppressWarnings({
        library(tidyverse)
        library(repr)
        library(tidymodels)
        library(GGally)
        library(gridExtra)
        library(grid)
        library(tidyr)
    })
})


In [4]:
# Size and general style set up
options(repr.plot.width = 6, repr.plot.height = 4, repr.matrix.max.rows = 7,readr.show_col_types = FALSE)

# Load Data
player_data <- read_csv("")
session_data <- read_csv("")



ERROR: Error: '' does not exist in current working directory ('/home/jovyan/work/player-subscription-ml/player-subscription-ml').


# Data Science Project: Project Final Report


## Introduction:

## Methods & Results:

### Set up and intro to data: "TODO should rename later." 

### Training and Analysis:

### Player Type Exploration: "TODO may need renaming"

## Discussion: